In [1]:
import threading
import optparse

import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.losses import mean_squared_error, logcosh, mean_absolute_error
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard

from model import mobnet

class ThreadSafeIterator:
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()

def threadsafe_generator(f):
    """
    A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*args, **kwargs):
        return ThreadSafeIterator(f(*args, **kwargs))

    return g

@threadsafe_generator
def train_generator(df, shape):
    while True:
        shuffle_indices = np.arange(len(df))
        shuffle_indices = np.random.permutation(shuffle_indices)
        
        for start in range(0, len(df), BATCH_SIZE):
            end = min(start + BATCH_SIZE, len(df))
            df_batch = df.iloc[shuffle_indices[start:end]]
            
            x_batch = []
            y_batch = []
            
            xcam = df_batch['XCam'].values
            ycam = df_batch['YCam'].values
            
            for index, _fn in enumerate(df_batch['file_names']):
                img = cv2.imread('{}/{}'.format(dset_path, _fn))
                img = cv2.resize(img, (shape[1], shape[0]), interpolation=cv2.INTER_AREA)

#               # === You can add data augmentations here. === #
#                 if np.random.random() < 0.5:
#                     img, mask = img[:, ::-1, :], mask[..., ::-1, :]  # random horizontal flip
                y_batch.append([xcam[index], ycam[index]])
                x_batch.append(img)
            
            yield np.asarray(x_batch), np.asarray(y_batch)

Using TensorFlow backend.


In [2]:
dset_path = '../gazecapture'

shape = (667, 375, 3)
lr = 1e-3
optim = RMSprop(lr)
loss = logcosh
BATCH_SIZE = 16

test = pd.read_csv('portrait-test-df.csv')
model = mobnet(shape, None)
model.compile(loss = loss,
            optimizer = optim,
            metrics = ['mae', 'mse'])


model.load_weights('logcosh-0.001-RMSprop-weights.hdf5')

metrics = model.evaluate_generator(generator=train_generator(test, shape),
                         steps=np.ceil(float(len(test)) / float(BATCH_SIZE)),
                         verbose=1)

5060/5060 [==============================] - 818s 162ms/step


In [3]:
model.metrics_names, metrics

(['loss', 'mean_absolute_error', 'mean_squared_error'],
 [0.8953140369502124, 1.3845042379869739, 3.937567944565428])